In [13]:
import pandas as pd
import pyrankvote
from pathlib import Path

In [14]:
df = pd.read_csv(Path('./data/20200720.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice],What theme should we explore next week? [Sixth Choice]
0,7/15/2020 18:08:51,Your Name.,UHF,The Night of the Hunter,The Spirit of the Beehive,The Secret Garden,EUROTRASH,I'M FUCKING ITALIAN OVER HERE,TIME TRAVEL,UNRELIABLE NARRATOR,SPORTS!,No Theme/Free-for-all
1,7/15/2020 18:23:44,The Spirit of the Beehive,Your Name.,UHF,The Secret Garden,NaN,I'M FUCKING ITALIAN OVER HERE,No Theme/Free-for-all,UNRELIABLE NARRATOR,SPORTS!,EUROTRASH,TIME TRAVEL
2,7/15/2020 18:42:25,UHF,The Spirit of the Beehive,Your Name.,The Night of the Hunter,The Secret Garden,SPORTS!,TIME TRAVEL,No Theme/Free-for-all,UNRELIABLE NARRATOR,EUROTRASH,I'M FUCKING ITALIAN OVER HERE
3,7/15/2020 18:43:05,Your Name.,UHF,The Night of the Hunter,The Spirit of the Beehive,The Secret Garden,TIME TRAVEL,UNRELIABLE NARRATOR,I'M FUCKING ITALIAN OVER HERE,SPORTS!,EUROTRASH,No Theme/Free-for-all
4,7/15/2020 19:13:59,Your Name.,UHF,The Secret Garden,NaN,NaN,TIME TRAVEL,SPORTS!,I'M FUCKING ITALIAN OVER HERE,EUROTRASH,No Theme/Free-for-all,UNRELIABLE NARRATOR
5,7/15/2020 19:19:59,The Night of the Hunter,UHF,NaN,NaN,NaN,EUROTRASH,TIME TRAVEL,UNRELIABLE NARRATOR,SPORTS!,I'M FUCKING ITALIAN OVER HERE,NaN
6,7/15/2020 19:25:54,The Night of the Hunter,Your Name.,UHF,The Secret Garden,The Spirit of the Beehive,UNRELIABLE NARRATOR,I'M FUCKING ITALIAN OVER HERE,TIME TRAVEL,SPORTS!,EUROTRASH,No Theme/Free-for-all
7,7/15/2020 20:55:03,Your Name.,UHF,The Secret Garden,NaN,NaN,UNRELIABLE NARRATOR,I'M FUCKING ITALIAN OVER HERE,SPORTS!,NaN,NaN,NaN
8,7/15/2020 23:30:40,The Night of the Hunter,The Spirit of the Beehive,The Secret Garden,UHF,Your Name.,I'M FUCKING ITALIAN OVER HERE,EUROTRASH,UNRELIABLE NARRATOR,SPORTS!,TIME TRAVEL,No Theme/Free-for-all
9,7/16/2020 11:32:53,Your Name.,The Night of the Hunter,UHF,The Secret Garden,The Spirit of the Beehive,I'M FUCKING ITALIAN OVER HERE,TIME TRAVEL,EUROTRASH,UNRELIABLE NARRATOR,NaN,NaN


In [15]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [16]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Your Name.', 'UHF', 'The Night of the Hunter',
       'The Spirit of the Beehive', 'The Secret Garden'], dtype=object)

In [17]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [18]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [19]:
submission_dict

{'Your Name.': <Candidate('Your Name.')>,
 'UHF': <Candidate('UHF')>,
 'The Night of the Hunter': <Candidate('The Night of the Hunter')>,
 'The Spirit of the Beehive': <Candidate('The Spirit of the Beehive')>,
 'The Secret Garden': <Candidate('The Secret Garden')>}

In [20]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [21]:
ballots

[<Ballot(Your Name., UHF, The Night of the Hunter, The Spirit of the Beehive, The Secret Garden)>,
 <Ballot(The Spirit of the Beehive, Your Name., UHF, The Secret Garden)>,
 <Ballot(UHF, The Spirit of the Beehive, Your Name., The Night of the Hunter, The Secret Garden)>,
 <Ballot(Your Name., UHF, The Night of the Hunter, The Spirit of the Beehive, The Secret Garden)>,
 <Ballot(Your Name., UHF, The Secret Garden)>,
 <Ballot(The Night of the Hunter, UHF)>,
 <Ballot(The Night of the Hunter, Your Name., UHF, The Secret Garden, The Spirit of the Beehive)>,
 <Ballot(Your Name., UHF, The Secret Garden)>,
 <Ballot(The Night of the Hunter, The Spirit of the Beehive, The Secret Garden, UHF, Your Name.)>,
 <Ballot(Your Name., The Night of the Hunter, UHF, The Secret Garden, The Spirit of the Beehive)>]

In [22]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [23]:
print(election_result)

FINAL RESULT
Candidate                    Votes  Status
-------------------------  -------  --------
Your Name.                       5  Elected
The Night of the Hunter          3  Rejected
UHF                              1  Rejected
The Spirit of the Beehive        1  Rejected
The Secret Garden                0  Rejected



In [24]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                        Votes  Status
-----------------------------  -------  --------
I'M FUCKING ITALIAN OVER HERE        3  Hopeful
TIME TRAVEL                          2  Hopeful
UNRELIABLE NARRATOR                  2  Hopeful
EUROTRASH                            2  Hopeful
SPORTS!                              1  Rejected
No Theme/Free-for-all                0  Rejected

ROUND 2
Candidate                        Votes  Status
-----------------------------  -------  --------
I'M FUCKING ITALIAN OVER HERE        3  Hopeful
TIME TRAVEL                          3  Hopeful
UNRELIABLE NARRATOR                  2  Hopeful
EUROTRASH                            2  Rejected
SPORTS!                              0  Rejected
No Theme/Free-for-all                0  Rejected

ROUND 3
Candidate                        Votes  Status
-----------------------------  -------  --------
I'M FUCKING ITALIAN OVER HERE        4  Hopeful
TIME TRAVEL                          4  Hopeful
UNRELIABL